In [ ]:
import pandas as pd
import numpy as np
import sktime
import seaborn as sns
import matplotlib.pyplot as plt
from convertcsv.import_preprocess_v4 import readcsvs
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sktime.transformations.panel.rocket import Rocket
from sktime.transformations.panel.rocket import MiniRocketMultivariate
from sktime.classification.kernel_based import RocketClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.impute import KNNImputer
from convertcsv.get_all_metrics_with_tags import get_all_metrics_with_tags

